# Advanced Notebook - Working with PUMS Microdata

CP201A - Fall 2025

# Learning Objectives

* Understand the PUMS dataset
* Load and clean PUMS data from CSV files
* Recategorize a categorical variable
* Clean a numeric variable
* Compute summary statistics from microdata
* Handle weighted observations
* Merge the housing units and persons tables

# What is PUMS?

Per the [Census Bureau](https://www.census.gov/programs-surveys/acs/microdata.html):

>The Census Bureau’s American Community Survey (ACS) Public Use Microdata Sample (PUMS) files enable data users to create custom estimates and tables, free of charge, that are not available through ACS pretabulated data products.  The ACS PUMS files are a set of records from individual people or housing units, with disclosure protection enabled so that individuals or housing units cannot be identified.

So, in short, while we've been working with pre-computed summary tables (each row summarizes data for a given geography) up until this point, now we will be working with the underlying microdata (each row represents one person or one housing unit).

Much more info in the very helpful [ACS PUMS Handbook](https://www.census.gov/content/dam/Census/library/publications/2021/acs/acs_pums_handbook_2021.pdf).

# Getting PUMS data

There are three ways to access PUMS data:
1. Download statewide housing units and persons CSV tables via File Transfer Protocol (FTP)
2. Prepare a custom extract/summary table using the Census Bureau's Microdata Access Tool (MDAT)
3. Pull specified variables from the Census Data API
More info and links for all three options can be found [here](https://www.census.gov/programs-surveys/acs/microdata/access.html).

The Microdata Access Tool is pretty cool and worth a look, but it's similar to data.census.gov as a whole in that it is probably too complicated for casual users and too oversimplified for power users. It *is* possible to pull PUMS data from the Census Data API, but it's not so easy to do so. The `census` package doesn't support this dataset (it works only for ACS and the decennial census), but you can still get the data by querying the relevant URL directly. I provide a quick example of how to do so at the end of this notebook, in case you want to do similar things in future; but **we're mostly going to work with PUMS data from CSV files today.** https://www.census.gov/programs-surveys/acs/microdata.html

## Figuring out what variables and geographies we're interested in

PUMS data can be a little overwhelming to work with at first, because there are so many variables and because we can't use the same geographies (counties, places, census tracts) that we use for tabulated ACS data. Instead, PUMS data are associated with Public Use Microdata Areas (PUMAs), which are contiguous areas composed of counties and census tracts with at least 100,000 population. 

The best data dictionary for PUMS variables is actually the `variables` page from the Census Data API: https://api.census.gov/data/2022/acs/acs5/pums/variables.html To check whether PUMS can help with your research question, you might start with this concise list of [topics covered in PUMS data](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/subjects_in_pums/2022_5yr_PUMS_Subjects.pdf); then I like to go to the Census Data API page and start searching for keywords. There are about 500 variables, so it can take some time to confirm that you've identified the most relevant variables for your question. Today we've made things simpler by providing an extract of certain variables.

You can use the [TIGERweb](https://tigerweb.geo.census.gov/tigerweb/) interactive web map to check PUMA boundaries and identify which PUMAs you're interested in. But note that PUMA identifiers (e.g. `07510`) are not unique across states, so you need to combine state FIPS codes (`06` for California) and PUMA IDs (`07510`, part of San Francisco city and county) to uniquely identify a PUMA. In low-population areas, PUMAs often encompass multiple counties, but in denser regions like the Bay Area, populous counties are often represented by multiple PUMAs. PUMA IDs consist of five numbers, the first three of which correspond to the three-digit FIPS code of the PUMA's (primary) county. We can take advantage of this format to quickly select data from specific counties.

## Reading in PUMS data from CSV files

Reading in a CSV as a pandas DataFrame is easy: `df = pd.read_csv('filename.csv')`. This is often all that's needed, but if you're dealing with large tables, it is sometimes very useful to specify additional arguments to `read_csv` to speed things up and preserve data quality:
* `nrows` lets us read in only that many rows. Something like `nrows=5` can be very helpful if you just want to take a quick look at which columns are in a given file.
* `usecols` tells pandas which columns to read in. The full PUMS CSVs are huge and reading in all columns is time- and memory-intensive, so specifying only the columns we need can be a good idea. (This argument isn't actually necessary for today's CSV files, which I've pre-trimmed down to the variables we need, but you should be familiar with this option for when you start working with really big tables.)
* `dtype` lets us tell pandas which datatypes to use for specific columns. We'll use this now to prevent pandas from misinterpreting the state and PUMA FIPS codes as numbers, when they're actually string-y IDs.

There are many other optional arguments for `pd.read_csv()` and many other functions and methods we've been using. How can you learn about them? **Pro Jupyter notebook coding tip**: you can check the documentation for a given function or method by pressing Shift-Tab inside the parentheses. So you can type something like `pd.crosstab(` and then press Shift-Tab and Jupyter will show you lots of useful information about this function. Quicker and more convenient than googling the function's name. 

Looking up information like this, or using Google or ChatGPT to figure out how to do things, is *completely normal*, by the way, and you will find yourself doing so frequently for the rest of your programming life. I looked up many dozens of things while writing these labs.

### Variables from the housing units table

We have provided the following variables from the full PUMS housing units table. You can click a variable's name to see its data dictionary from the Census Data API.

| Topic | Variable/Documentation | Label |
|-|-|-|
| Geography | [ST](https://api.census.gov/data/2022/acs/acs5/pums/variables/ST.json) | State code |
| Geography | [PUMA10](https://api.census.gov/data/2022/acs/acs5/pums/variables/PUMA10.json) | PUMA code based on 2010 Census definition for data years 2012-2021 |
| Geography | [PUMA20](https://api.census.gov/data/2022/acs/acs5/pums/variables/PUMA20.json) | PUMA code based on 2020 Census definition for data years 2022 and later |
| Unique ID | [SERIALNO](https://api.census.gov/data/2022/acs/acs5/pums/variables/SERIALNO.json) | Housing unit/GQ person serial number |
| Weighting | [WGTP](https://api.census.gov/data/2022/acs/acs5/pums/variables/WGTP.json) | Housing Unit Weight |
| Building age | [YRBLT](https://api.census.gov/data/2022/acs/acs5/pums/variables/YRBLT.json) | When structure first built |
| Building type | [BLD](https://api.census.gov/data/2022/acs/acs5/pums/variables/BLD.json) | Units in structure |
| Housing tenure (rent/own) | [TEN](https://api.census.gov/data/2022/acs/acs5/pums/variables/TEN.json) | Tenure |
| Housing cost | [GRNTP](https://api.census.gov/data/2022/acs/acs5/pums/variables/GRNTP.json) | Gross rent (monthly amount) |
| Housing cost adjustment | [ADJHSG](https://api.census.gov/data/2022/acs/acs5/pums/variables/ADJHSG.json) | Adjustment factor for housing dollar amounts |
| Household income | [HINCP](https://api.census.gov/data/2022/acs/acs5/pums/variables/HINCP.json) | Household income (past 12 months) |
| Household income adjustment | [ADJINC](https://api.census.gov/data/2022/acs/acs5/pums/variables/ADJINC.json) | Adjustment factor for income and earnings dollar amounts |
| Rent burden | [GRPIP](https://api.census.gov/data/2022/acs/acs5/pums/variables/GRPIP.json) | Gross rent as a percentage of household income past 12 months |

I've provided a code snippet below that reads in the housing units table, using `usecols` and `dtype` to fine-tune the command.

In [ ]:
hhs = pd.read_csv('psam_h06.csv')
print(hhs.columns.tolist())

In [ ]:
import pandas as pd

hhs = pd.read_csv(
    'psam_h06.csv',
    usecols=['STATE',
        'PUMA',
        'SERIALNO',
        'WGTP',
        'YRBLT',
        'BLD',
        'TEN',
        'GRNTP',
        'ADJHSG',
        'HINCP',
        'ADJINC',
        'GRPIP',
    ],
    dtype={'PUMA': str,
    },
)
hhs

In [ ]:
sorted(hhs['PUMA'].unique())

In [ ]:
# Now we can select data for SF in CA
hhs = hhs[(hhs['STATE'] == 6) & (hhs['PUMA'].str[:3].isin(['075']))]

# Check how many rows we have remaining
len(hhs)

In [ ]:
persons = pd.read_csv(
    'psam_p06.csv',
    usecols=[
        'STATE',
        'PUMA',
        'SERIALNO',
        'SPORDER',
        'PWGTP',
        'RAC1P',
        'HISP',
    ],
    dtype={'PUMA': str,
    },
)

persons = persons[(persons['STATE'] == 6) & (persons['PUMA'].str[:3].isin(['075']))]

# Recoding categorical variables

`BLD` (units in structure) shows up looking like an integer. Does this field simply tell us the number of units in the building? Let's imagine for a moment that we are sloppy analysts who don't read the documentation first. (We've all been there.) One way we could try to check whether this is what `BLD` is truly telling us is to plot a histogram of `BLD` and see if it looks reasonable:

In [ ]:
# This is just a quick and dirty histogram for a sanity check
hhs['BLD'].plot.hist()

In [ ]:
# And we can use .value_counts() to see a tabular representation of the frequencies
# dropna=False because we also want to inspect missingness
hhs['BLD'].value_counts(dropna=False)

Wait a minute. The number of units in structures maxes out below 10? The large majority of structures contain two units?? This looks very wrong.

OK, now let's do what we should have done in the first place: take a look at the [API reference page](https://api.census.gov/data/2022/acs/acs5/pums/variables/BLD.json) for `BLD`. Aha! `BLD` is actually a *categorical* variable. Each "integer" value corresponds to a category. We can clean this up substantially, but first, let's look into those `NaN` values...

## Handling missing values

One difference between the data dictionary from the Census Data API and the CSV version of the PUMS data is that the data dictionary says `0` corresponds to "N/A (GQ)" (i.e., the "household" is in group quarters so the question doesn't apply), but we can see via `.value_counts()` above that there are no zeros, only `NaN` ("not a number," i.e. missing data). We should think about how we want to handle missingness. We could drop records with missing data: [`df.dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html). Or we could fill missing data with an appropriate value: [`df.fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html). This latter approach is more appropriate here, so let's fill missing values in the `BLD` column with zeros:

In [ ]:
# We should first check whether all missing values occur in group-quarters rows.
# We can do so with pd.crosstab(), a very handy tool both for quick diagnostics
# and for more nuanced analyses
pd.crosstab(
    hhs['SERIALNO'].str.contains('GQ'),
    hhs['BLD'].isna()
)

In [ ]:
# Great, we've confirmed that only GQ rows have missing BLD values.
# Now we can use .fillna() to fix this missingness! 
# As an added bonus, filling missing values allows us to convert BLD from floats to ints.
# (Why was it float in the first place? Because NaN *is* a float:
# https://stackoverflow.com/questions/48558973/why-is-nan-considered-as-a-float )
hhs['BLD'] = hhs['BLD'].fillna(0).astype(int)
hhs.head()

# You could probably apply similar treatments to several other columns in our DataFrame...

Let's *recode* `BLD` to group the default categories into groups that are meaningful for our interests/needs. This is easy to do in pandas, using the `.map()` method. You may recall this method from the extra materials from last week, where we created a dict that mapped census tract FIPS codes to neighborhood names. Here, we'll specify a mapping from integer values of `BLD` to meaningful categories.

Let's group `BLD` into four categories:
1. Single-family houses
2. Small multifamily buildings (2-19 units)
3. Large multifamily buildings (20 or more units)
4. Other

I've started you off with a dict containing the default meanings of each value of `BLD`. (I copy-pasted this from the BLD.json link above, then lightly cleaned it up by converting string keys to ints and sorting the keys.) Try editing `recode_BLD` to map the integer values into the four categories we want. 

In [ ]:
recode_BLD = {
    0: "N/A (GQ)",
    1: "Mobile Home or Trailer",
    2: "One-family house detached",
    3: "One-family house attached",
    4: "2 Apartments",
    5: "3-4 Apartments",
    6: "5-9 Apartments",
    7: "10-19 Apartments",
    8: "20-49 Apartments",
    9: "50 or More Apartments",
    10: "Boat, RV, van, etc.",
}
recode_BLD = {
    0: "Other",
    1: "Other",
    2: "Single-family house",
    3: "Single-family house",
    4: "Small multifamily building (2-20 units)",
    5: "Small multifamily building (2-20 units)",
    6: "Small multifamily building (2-20 units)",
    7: "Small multifamily building (2-20 units)",
    8: "Large multifamily building (20 or more units)",
    9: "Large multifamily building (20 or more units)",
    10: "Other",
}

# Let's save the recoded variable into a new column.
# .astype('category') takes advantage of pandas' Categorical data type:
# https://pandas.pydata.org/docs/user_guide/categorical.html
hhs['building_type'] = hhs['BLD'].map(recode_BLD).astype('category')
hhs['building_type'].value_counts()

While we're at it, let's clean up `TEN` too. Check the [API data dictionary](https://api.census.gov/data/2022/acs/acs5/pums/variables/TEN.json), then try filling missingness appropriately and mapping ints to meaningful labels, in a new `tenure` column:

In [ ]:
recode_TEN = {
    0: "N/A (GQ/vacant)",
    1: "Owned with mortgage or loan (include home equity loans)",
    2: "Owned Free And Clear",
    3: "Rented",
    4: "Occupied without payment of rent",
}

hhs['tenure'] = hhs['TEN'].fillna(0).map(recode_TEN).astype('category')
hhs['tenure'].value_counts()

## Combining two variables to form a new one

Our `persons` table has a variable about race (`RAC1P`) and a variable about ethnicity (`HISP`). Let's create a new variable that expresses both race and ethnicity via five categories: NH White, NH Black, NH Asian, NH Other, and Hispanic.

As usual, a good starting point is to check the data dictionaries for [`RAC1P`](https://api.census.gov/data/2022/acs/acs5/pums/variables/RAC1P.json) and [`HISP`](https://api.census.gov/data/2022/acs/acs5/pums/variables/HISP.json).

In [ ]:
# We'll start by collapsing categories in RAC1P to just four
persons['race'] = persons['RAC1P'].map({
    1: 'White',
    2: 'Black',
    3: 'Other',
    4: 'Other',
    5: 'Other',
    6: 'Asian',
    7: 'Asian',  # or Other, arguably
    8: 'Other',
    9: 'Other',
})

# Then create race_ethnicity based on race
persons['race_ethnicity'] = 'NH ' + persons['race']
# And overwrite rows for Hispanic respondents
persons.loc[persons['HISP'] != 1, 'race_ethnicity'] = 'Hispanic'
# Convert this column to pd.Categorical
persons['race_ethnicity'] = persons['race_ethnicity'].astype('category')

persons['race_ethnicity'].value_counts()

# Cleaning numeric variables

Let's take a look at some numeric variables in our dataset: `GRNTP`, meaning gross monthly rent, and `HINCP`, meaning household income over the past 12 months. We want to *clean* these variables: understand and potentially address any missingness, and understand and potentially address outlier values.

## Check missingness

Checking the [`GRNTP` data dictionary](https://api.census.gov/data/2022/acs/acs5/pums/variables/GRNTP.json), we see that a value of 0 means "N/A (GQ/vacant/owned or being bought/occupied without rent payment)". But one difference between the Census Data API version of PUMS data and the CSV version is that "N/A" values are often just empty (null) cells, so we should inspect our variables both for `0`s and `NaN`s (null values). 

We can quickly check the percentage of rows with missing values, as well as with 0s in the `GRNTP` column:

In [ ]:
# Percent of rows with missing values. .isna() identifies rows with missingness,
# then .sum() converts True to 1 and False to 0 and adds up all the 1s.
# Then divide by the full DF length to get the share that are null
print(hhs['GRNTP'].isna().sum() / len(hhs))

# Similar logic for calculating the percent of rows with 0 as the value
print((hhs['GRNTP'] == 0).sum() / len(hhs))

Hmm, OK, no zeros, but 62 percent of rows are missing values. Can we explain why this is the case? The data dictionary suggests that many "N/A"s are associated with housing units that are owned (rather than rented), vacant, or in group quarters. We can calculate a cross-tab with `tenure`, which we cleaned up earlier, to see whether this substantially explains the missingness in `GRNTP`:

In [ ]:
# "True" refers to rows where GRNTP is missing (NaN)
pd.crosstab(hhs['GRNTP'].isna(), hhs['tenure'])

Ah, very good. We can see that all the missingness in `GRNTP` (gross rent) occurs when `tenure` is not "rented". In this case, null values in `GRNTP` are appropriate, although you might consider setting gross rent to 0 for those few hundred lucky respondents occupying units without paying rent.

## Check outliers

Another important step in cleaning numeric data is identifying outliers and handling them appropriately.

There is no one hard-and-fast rule about identifying outliers. Sometimes you may wish to retain the inner 98 percent of the distribution (omitting the top and bottom percent of the data); other times you may want to drop observations outside some multiple of the [interquartile range](https://online.stat.psu.edu/stat200/lesson/3/3.2) (this is sometimes called a "[Tukey](https://en.wikipedia.org/wiki/John_Tukey) fence"); sometimes certain values will be facially unreasonable, like negative values for headcounts or percentages greater than 100%. Ultimately, though, because the object of outlier detection is to identify bad or un-useful data, there is no substitute for careful consideration of relevant domain knowledge.

Pandas' `.describe()` method is a good place to start, as it gives us a sense of the range and central tendency of the variables' distribution:

In [ ]:
hhs[['GRNTP', 'HINCP']].describe()

Note that the maximum value for `GRNTP` (monthly gross rent) is about 3 times the 75th percentile value, but the maximum value for `HINCP` (annual household income) is *ten times* the 75th percentile value. This already suggests that `HINCP` is likelier to have some outliers than `GRNTP`. 

Let's take a look at the distributions of `GRNTP` and `HINCP`, always a good idea when taking the temperature of a variable:

In [ ]:
hhs['GRNTP'].plot.hist(bins=30)

In [ ]:
hhs['HINCP'].plot.hist(bins=30)

That "second hump" above \\$6000 for `GRNTP` looks a little suspicious, but rents above \\$6000 are (sadly) not unheard of in the Bay Area. You might want to mark anything about \\$6000 as outliers, or not. (Stretch goal for later: can you use other PUMS variables, like [`BDSP`](https://api.census.gov/data/2022/acs/acs5/pums/variables/BDSP.json), to test whether these higher rents really are plausible?) Similarly, the `HINCP` distribution shows a few extremely high-earning households. We might cut things off at \\$1,000,000. Think about it and decide what you think is reasonable.

Think also about how to handle outliers. There are three main ways:
1. Bottom/top-coding or "clipping" outliers - overwriting values outside a specified range with the min/max of that range. You can use pandas' [`.clip()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.clip.html) method for this.
2. Replacing outliers with nulls / `NaN` - this will exclude the rows in question from analyses pertaining to this variable, but will retain the rest of their data. You can set the relevant cells to `np.nan`.
3. Dropping rows with outliers entirely - this will exclude the rows in question from *all* analyses. You already know how to select rows of a DataFrame based on numeric criteria.

Once you've decided what to call outliers and how to handle them, try implementing your approach below:

In [ ]:
# I will simply set HINCP > 1000000 to NaN
import numpy as np
hhs.loc[hhs['HINCP'] > 1000000, 'HINCP'] = np.nan

## Adjusting dollar variables for inflation

The 5-year PUMS data contain observations collected between 2018 and 2022. The real value of the dollar was fluctuating over this time period, so PUMS includes adjustment factors to convert amounts from different years into comparable real dollars. As noted in the [API reference for `HINCP`](https://api.census.gov/data/2022/acs/acs5/pums/variables/HINCP.json), we should use `ADJINC` to adjust the values in `HINCP`. Other variables (like `GRNTP`) use `ADJHSG` instead; check the documentation!

`ADJINC` and `ADJHSG` are both integers that implicitly represent floating-point adjustment factors with six decimal points:

In [ ]:
# Fun fact: there is only one value of ADJINC for each year in the dataset
print(hhs['ADJINC'].value_counts())

# We need to convert these integers into floating-point numbers to use them
hhs['ADJINC'] = hhs['ADJINC'] / 1000000

# Now we can adjust HINCP
hhs['HINCP'] = hhs['HINCP'] * hhs['ADJINC']

hhs['HINCP'].describe()

## Binning numeric variables into categories/ranges

We can convert quasi-continuous variables like `HINCP` into discrete, categorical variables using [`pd.cut()`](https://pandas.pydata.org/docs/reference/api/pandas.cut.html):

In [ ]:
hhs['income_bin'] = pd.cut(
    hhs['HINCP'],
    [-99999, 0, 50000, 100000, 200000, 500000, 9999999],
    labels=[
        '$0 or negative',
        '$1-$50,000',
        '$50,001-$100,000',
        '$100,001-$200,000',
        '$200,001-$500,000',
        'More than $500,000'
    ]
)

hhs['income_bin'].value_counts()

# Computing summary statistics from microdata

Now that we have cleaned our numeric variables, let's calculate summary statistics for them!

The *mean* (average) is easy:

In [ ]:
# We could compute the mean manually...
hhs['HINCP'].sum() / hhs['HINCP'].count()

# ...or just use the pandas built-in method
hhs['HINCP'].mean()

Variance is a little more complex:

$$ Var(X) = \frac{1}{n - ddof} \sum_{i=1}^n{(x_i - \mu)^2},$$

where $\mu$ is the mean of $X$, and $ddof$ (delta degrees of freedom) is 0 for population variance or 1 for [sample variance](https://en.wikipedia.org/wiki/Bessel's_correction).

In [ ]:
# We could compute variance manually...
((hhs['HINCP'] - hhs['HINCP'].mean())**2).sum() / (hhs['HINCP'].count() - 1)

# ...or just use the pandas built-in method
hhs['HINCP'].var()

Standard deviation is just the square root of variance:

In [ ]:
# We could compute standard deviation manually...
hhs['HINCP'].var()**0.5

# ...or just use the pandas built-in method
hhs['HINCP'].std()

The [standard error of the mean](https://en.wikipedia.org/wiki/Standard_error) is equal to the sample standard deviation $\sigma_x$ divided by the square root of the sample size:

$$\sigma_\bar{x} = \frac{\sigma_x}{\sqrt{n}}$$

In [ ]:
# We could compute the standard error of the mean manually...
hhs['HINCP'].std() / hhs['HINCP'].count()**0.5

# ...or just use the pandas built-in method
hhs['HINCP'].sem()

We can derive margins of error for various confidence levels by multiplying the standard error by the appropriate z-value (1.645 for 90% confidence level, 1.960 for 95% confidence level, 2.576 for 99% confidence level):

In [ ]:
# 95% confidence level margin of error
hhs['HINCP'].sem() * 1.96

Finally, the median is also easy to calculate using a pandas built-in method:

In [ ]:
hhs['HINCP'].median()

## Handling weighted observations

But there's a catch! You may have noticed the `WGTP` and `PWGTP` variables in the housing units and persons tables, respectively. These variables represent the *weights* associated with each record. The Census Bureau estimates these weights such that the PUMS records, if expanded accordingly, would amount to the total population of the area. Both the housing unit and person weights vary substantially, so our sample mean is inaccurate unless we account for them.

The formula for the [weighted mean](https://en.wikipedia.org/wiki/Weighted_arithmetic_mean) is:

$$\bar{x} = \frac{\sum_{i=1}^n{w_i x_i}}{\sum_{i=1}^n{w_i}},$$

where $w$ are the weights and $x$ are the variable's values. We can use [numpy's `average` function](https://numpy.org/doc/stable/reference/generated/numpy.average.html) to quickly perform this calculation:

In [ ]:
# The hardest part here is that we need to exclude rows where HINCP is NaN
np.average(
    hhs.loc[hhs['HINCP'].notna(), 'HINCP'],
    weights=hhs.loc[hhs['HINCP'].notna(), 'WGTP']
)

The weighted average household income is about $8,000 less than the unweighted average. Using the weights makes a real difference!

**Note:** *correctly* calculating standard errors for estimates based on PUMS microdata is technically complex and well beyond the scope of this lab. (If you're really interested, you can learn more in the technical documentation, specifically [PUMS Accuracy of the Data](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/accuracy/2018_2022AccuracyPUMS.pdf). It would be a nice stretch goal to use what you've learned in these labs to implement the Successive Difference Replication formula for calculating standard errors.) For today, we will stick with the *unweighted* variance, standard deviation, and standard error of the mean.

See [here](https://stackoverflow.com/questions/20601872/numpy-or-scipy-to-calculate-weighted-median) if you need to compute a weighted *median*, and want to see some slick numpy methods in action. 

# Merging the housing units and persons tables

Sometimes we want to consider both housing unit-specific and person-specific variables in our analyses. This means we need to *merge* the `hhs` and `persons` DataFrames.

Merging is a powerful, fundamental pandas operation, discussed in great detail in the pandas [user guide](https://pandas.pydata.org/docs/user_guide/merging.html), [tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html), and [API reference](https://pandas.pydata.org/docs/reference/api/pandas.merge.html). We'll just see a quick example today, but this is definitely a topic worthy of further exploration.

In [ ]:
# The basic syntax is left.merge(right, on=column or columns in common).
# df.merge() defaults to an *inner join*, where only keys that appear in both
# the left and right DataFrames will be retained. You can control this behavior
# with the `how` keyword argument. Note that df.join() (which merges on indices)
# defaults to a *left join*
hhs.merge(persons, on='SERIALNO')

We wind up with 122,950 rows, one for each row in `persons`. This is because every value of `SERIALNO` that appears in the `persons` table also appears in the `hhs` table. But *not* every `SERIALNO` that appears in `hhs` also appears in `persons`! Can you think why this is the case? (Hint: think about the various values of the `tenure` variable.)

We have also wound up with some unattractive and unwieldy `_x` and `_y` columns. This happens when the left and right DataFrames contain columns with the same names. We can avoid this by merging only a subset of the columns in `persons` onto `hhs`:

In [ ]:
merged = hhs.merge(persons[['SERIALNO', 'SPORDER', 'PWGTP', 'race_ethnicity']], on='SERIALNO')
merged

As a micro-capstone, let's use this merged table to show a cross-tabulation of some housing unit-related variables and some person-related variables. I'll provide an example; try to adapt it to examine some interactions you're curious about:

In [ ]:
pd.crosstab(merged['income_bin'], merged['race_ethnicity'])

In [ ]:
# We can check (unweighted) median household income by race/ethnicity
merged.groupby('race_ethnicity')['HINCP'].median()

In [ ]:
# And we can calculate *weighted* mean household income by race/ethnicity of head of household,
# using *lambda functions*, which are small anonymous functions that are really worth learning about

# We select SPORDER == 1 to analyze only heads of households
merged[(merged['SPORDER'] == 1) & merged['HINCP'].notna()].groupby('race_ethnicity').apply(lambda df: np.average(df['HINCP'], weights=df['WGTP']))

# Saving your data (in a better format than CSV)

Before we wrap up, let's save our data tables to disk. We could use `df.to_csv()` to write them to CSVs, but now is a good time to try out a different, [arguably superior](https://www.analyticsvidhya.com/blog/2022/06/stop-storing-data-in-csvs-vs-feather/) file format: `.feather` files. Feather files are usually smaller than CSVs, are dramatically faster to read and write (this is important as our tables get larger and larger), and can preserve datatypes like `pd.Categorical` and datetimes. They're also stable across multiple versions of pandas and intercompatible with R and other data science tools. Their only real disadvantage is that you can't open them in Notepad/Excel and manually inspect their contents.

You'll need to `conda install pyarrow` (feather files are part of the [Apache Arrow](https://arrow.apache.org/) project) into your conda environment. Restart the kernel and rerun all cells in this notebook if necessary, then you can `pd.read_feather()` and `df.to_feather()` to your heart's content.

In [ ]:
hhs.to_feather('hhs.feather')
persons.to_feather('persons.feather')
merged.to_feather('merged.feather')

# Bonus example: Pulling PUMS data from the Census Data API

The below cell shows how to use the `requests` library to pull PUMS data directly from the Census Data API. 

A few notes:
* It's a good idea to always pull the variables `SERIALNO` (unique housing unit identifier) and `WGTP` (housing unit weight), as well as `SPORDER` (person number within household) and `PWGTP` (person weight) if you're pulling person-related variables; these let us uniquely identify each housing unit and person and weight their data appropriately.
* You can pull housing unit-related *and* person-related variables in the same Census Data API request, as demonstrated by our pulling `BDSP` (number of bedrooms, a housing unit variable) and `SEX` (individual's sex, a person variable). The two tables come pre-merged.
    * All persons in the same housing unit will share that housing unit's values for all housing unit-related variables, so if you're interested in analyzing housing unit-specific attributes, make sure to select only rows where `SPORDER == '01'` (i.e. heads of household) so you don't double-count housing units occupied by multiple persons. Or much better yet, make sure your data pull includes only housing unit-related variables so that the resulting table will include *all* housing units, even vacant ones (which would otherwise be implicitly discarded during the Census Data API's automatic [inner join](https://www.w3schools.com/sql/sql_join.asp) of housing units and persons).
* Selecting geographies is tricky. You can't pass a comma-delimited list of PUMAs like you can do with census tracts; the finest geographic scale you can specify is states, like `for=state:06`. But you can use *predicates* (essentially selection criteria) to pull data for one PUMA at a time (`PUMA20=xxxxx`). You can't pull multiple PUMAs at once this way, so it may be faster to download all relevant data statewide and then select PUMAs afterwards.
* You can also use predicates to pull only records matching certain criteria (e.g. `MAR=1` means "return only married individuals").

In [ ]:
import requests
import pandas as pd

# We'll structure our helper lists/dicts slightly differently than before.
# First we'll write a list of variables for which to pull data
variables_of_interest = [
    'SERIALNO',  # unique housing unit identifier
    'SPORDER',  # person number within household
    'WGTP',
    'PWGTP',
    'BDSP',
    'SEX',
]
# Then a list of *predicates*, which filter the data and also show up as columns
predicates = [
    'for=state:06',  # CA only
    'PUMA20=07510',  # PUMA 07510 only
    'MAR=1',  # married - https://api.census.gov/data/2022/acs/acs5/pums/variables/MAR.json
]
# And finally a dict to help us rename columns; we include only numeric
# columns here so we can also use this to batch-convert columns to ints
rename_dict = {
    'WGTP': 'hh_weight',  # household weight
    'PWGTP': 'person_weight',  # person weight
    'BDSP': 'num_bedrooms',  # number of bedrooms (household attribute)
    'SEX': 'sex',  # person's sex (1 = male; 2 = female),
    'MAR': 'marital_status',  # 1 = married
}

# Now we prepare our API query URL using an f-string and the str.join() method
api_key = '827a8aafba255606ced9b44cd4380ba61ad39003'
query_url = f'https://api.census.gov/data/2022/acs/acs5/pums?get={','.join(variables_of_interest)}&key={api_key}&{'&'.join(predicates)}'
print(query_url)

# Send the API request and store the result in `r`
r = requests.get(query_url)

# Take a look at the first few items in the JSON representation of the resulting data
print(r.json()[:5])

# The 0th element of the JSON contains the column names, so separate that from the rest
cols = r.json()[0]
data = r.json()[1:]

# And now create a DataFrame using `data` and `cols`
df = pd.DataFrame(data, columns=cols)

# Convert some columns to ints
for col in rename_dict.keys():
    df[col] = df[col].astype(int)

# Rename some columns
df = df.rename(columns=rename_dict)

df